 📌 Nmap & Wireshark Email Analysis Project (Phase 1)

 **Introduction**
This project will analyze the transmission of emails between two devices on the same network:  
- **Dell OptiPlex (Windows 11, Chrome, steventuschman@gmail.com)**  
- **MacBook Air (macOS, Safari, steventuschman1@gmail.com)**  

Both devices will use **Gmail's web interface**, but each will send and receive emails from a **separate Gmail account**. This allows us to analyze how email transmission occurs under different conditions.

We will conduct **two email transmissions**:
1. **Email sent from the Dell (Chrome) to the MacBook (Safari)**  
   - **Sender**: steventushman@gmail.com (Chrome on Dell)  
   - **Recipient**: steventushman1@gmail.com (Safari on MacBook)  
2. **Email sent from the MacBook (Safari) to the Dell (Chrome)**  
   - **Sender**: steventushman1@gmail.com (Safari on MacBook)  
   - **Recipient**: steventushman@gmail.com (Chrome on Dell)  

By capturing **network traffic** during these email transmissions, we can compare **how each system communicates with Gmail’s servers** and analyze potential security and performance differences.

---

 **Project Goals**
- Use **Wireshark** to capture and analyze **email-related network traffic** for both transmissions.
- Use **Nmap** to scan and fingerprint Gmail’s email servers.
- Identify **DNS lookups**, **TLS handshakes**, and **email transmission metadata**.
- Compare how **Chrome on Windows vs. Safari on macOS** handles email transmission.
- Lay the groundwork for **Phase 2**, where we analyze email clients like Outlook and Thunderbird.

---

 **Tools & Methodology**
 **🛠️ Tools Used**
1. **Wireshark** - To capture network packets and analyze email transmission behavior.
2. **Nmap** - To scan and fingerprint email-related ports and services.
3. **Gmail Accounts** - Two separate Gmail addresses for bidirectional email testing.
4. **Web-Based Email Clients**:
   - **Chrome on Windows** (Dell, steventuschman@gmail.com)
   - **Safari on macOS** (MacBook, steventuschman1@gmail.com)

---

 **🔍 What We’ll Be Looking For**
 **1️⃣ Wireshark Analysis**
- **DNS Lookups**: Identifying how each device resolves Gmail’s mail server’s IP address.
- **TLS Handshake**: Analyzing how encryption is established between each device and Gmail’s servers.
- **HTTPS Traffic**: Observing how email content is transmitted securely within an encrypted session.
- **Session Establishment**: Capturing packet flow from email initiation to completion.
- **Differences Between Dell & MacBook Traffic**:
  - How **Chrome vs. Safari** handles Gmail connections.
  - Any variations in **TLS encryption** or **session setup time**.
  - Differences in **DNS caching behavior** between Windows and macOS.

 **2️⃣ Nmap Analysis**
- **Port Scanning**: Identifying which ports are used for Gmail’s email transmission.
- **Service Fingerprinting**: Analyzing the email servers’ protocols and security configurations.
- **SSL/TLS Enumeration**: Checking the cryptographic strength of Gmail’s mail servers.

---

 **Next Steps**
1️⃣ Capture a Wireshark session while sending an email from Dell (Chrome) to MacBook (Safari).
2️⃣ Filter and analyze the captured packets for DNS, TLS, and email-related traffic.
3️⃣ Run an Nmap scan against Gmail’s mail server (smtp.gmail.com) to identify its open ports and encryption settings.

📌 Now Reverse the Direction (MacBook to Dell) and Repeat

4️⃣ Capture a Wireshark session while sending an email from MacBook (Safari) to Dell (Chrome).
5️⃣ Filter and analyze the captured packets for DNS, TLS, and email-related traffic (same as Step 2).
6️⃣ Run an Nmap scan against Gmail’s mail server (smtp.gmail.com) again, but this time from the MacBook, to check for any variations in Gmail’s response.
7️⃣ Compare both sets of results to identify any network-level differences.



---

 **Future Expansion (Phase 2)**
- Set up a **Windows email client** (Outlook or Thunderbird) to send and receive emails.
- Capture **SMTP, IMAP, or MAPI traffic** for deeper protocol analysis.
- Compare **plaintext vs. encrypted email transmissions**.
- Investigate **authentication mechanisms** used in email clients.

---




 # **DNS Resolution Analysis: Identifying Gmail's Mail Server**
## **Objective**
Before Gmail's mail server could encrypt and transmit email data, the **Dell (Chrome) needed to determine the IP address of Gmail’s mail server (`mail.google.com`)**.  
This process, known as **DNS resolution**, involves querying a **DNS server** to translate a domain name (`mail.google.com`) into an IP address.

We used **Wireshark to capture and analyze this DNS lookup**, identifying the resolved IP address and confirming how Gmail’s infrastructure handles email routing.

---

## **Step 1: Capturing the DNS Query**
Using the **DNS filter (`dns`) in Wireshark**, we located the **DNS query sent by the Dell to resolve `mail.google.com`**.

- **Query Sent:** `mail.google.com`
- **Packet Identified:** **Frame 29**
- **Source IP (Dell):** `2600:1700:a3a0:d90:1c5c:202e:e1ff:75bd`
- **Destination IP (DNS Server):** `2600:1700:a3a0:d90::1`
- **Query Type:** AAAA (IPv6)

📌 **Observations:**  
- The DNS request was an **IPv6 query**, meaning that Gmail's mail infrastructure supports IPv6.  
- No IPv4 (A record) query was made, suggesting that Chrome on Windows **defaulted to IPv6**.

---

## **Step 2: Capturing the DNS Response**
After sending the query, the Dell **received a DNS response from the DNS server**, which provided the resolved IP address for Gmail's mail server.

- **Packet Identified:** **Frame 38**
- **Resolved IP Address (Gmail's Mail Server):** `2607:f8b0:4005:802::2005`
- **Response Time:** 0.004542 seconds
- **Query Type:** AAAA (IPv6)
- **Source IP (DNS Server):** `2600:1700:a3a0:d90::1`
- **Destination IP (Dell):** `2600:1700:a3a0:d90:1c5c:202e:e1ff:75bd`

📌 **Observations:**  
- The DNS query completed **in less than 5 milliseconds**, showing an efficient name resolution process.  
- Gmail provided an **IPv6 address**, reinforcing that Google's mail infrastructure prefers IPv6 connectivity.  
- No fallback to **IPv4 (A record) resolution** was observed.

---

## **Step 3: Implications of DNS Resolution**
### **1️⃣ Why DNS Resolution Matters for Email Transmission**
- Before an email can be sent, the **sender's computer must resolve the recipient's mail server address**.
- **DNS lookups introduce potential delays**, so Google's use of **fast IPv6 resolution optimizes email delivery.**
- The resolved **IP address (`2607:f8b0:4005:802::2005`) is part of Google’s global mail infrastructure**, meaning emails sent via Gmail route through Google's massive data center network.

### **2️⃣ What This Tells Us About Gmail’s Email Routing**
- Gmail’s email servers are distributed **across different IPs based on load balancing**.
- Even though we resolved **one IP (`2607:f8b0:4005:802::2005`)**, subsequent queries **could return different IPs** depending on network conditions.
- **DNS caching** may cause future email transactions to skip resolution if an IP is already known.

### **3️⃣ Security & Network Analysis Considerations**
- **DNS Spoofing Risks:** If an attacker manipulates DNS responses, they could redirect email traffic to a **malicious server**.
- **Encrypted DNS (DoH/DoT) Impact:** Some modern browsers use **DNS over HTTPS (DoH)**, which prevents capturing DNS queries in Wireshark. In this case, we were able to capture it because standard **UDP-based DNS resolution was used**.

---

## **Conclusion**
✅ **DNS resolution was successfully captured and analyzed.**  
✅ **Gmail's mail server was resolved to an IPv6 address (`2607:f8b0:4005:802::2005`).**  
✅ **The quick response time (0.0045s) confirms that Gmail optimizes domain resolution.**  
✅ **IPv6 preference suggests that Gmail prioritizes modern network infrastructure.**  

📌 **This DNS resolution process was the first critical step in Gmail’s email transmission. The next step was establishing encryption, which was handled via QUIC instead of traditional TLS over TCP.**


# **QUIC Handshake Analysis: Gmail Email Transmission**
## **Objective**
The goal of this analysis was to observe how encryption is established when sending an email from **Chrome on the Dell (Windows 11) to Gmail’s mail server**, using **Wireshark to capture and analyze traffic**.  
Initially, we expected to see a **TLS handshake** over **TCP (Port 443)**. However, after filtering the traffic, we discovered that **Gmail does not use traditional TLS over TCP for this transaction**. Instead, Gmail employs **QUIC**, a transport-layer protocol that combines encryption and session establishment directly over **UDP (Port 443)**.

---

## **Step 1: Identifying Gmail’s Mail Server**
Before encryption could begin, the Dell needed to **resolve Gmail’s mail server IP**.  
This was captured in the **DNS resolution process**:

- **Query:** `mail.google.com`
- **DNS Request:** **Frame 29**
- **DNS Response:** **Frame 38**
- **Resolved IP Address:** `2607:f8b0:4005:802::2005`
- **Record Type:** AAAA (IPv6)

This confirmed that **Gmail supports IPv6 connections for email transmission**.

---

## **Step 2: Identifying the QUIC Handshake**
Upon analyzing **TLS-filtered traffic**, we found that Gmail was **not using traditional TLS**.  
Instead, we saw **QUIC traffic**, indicating that encryption was being established through **QUIC instead of TLS over TCP**.

🔹 **Key QUIC Handshake Packets Identified:**
- **Frame 25** (Dell → Gmail)  
  - **QUIC Initial Packet**
  - **Crypto Frames present** (this replaces traditional "Client Hello")
  - **Destination Gmail Server:** `2607:f8b0:4005:810::2003`
  - **Sent over UDP 443**

- **Frame 26** (Dell → Gmail)  
  - **Continuation of QUIC handshake**
  - **Contains additional crypto and PING frames**

- **Frame 50** (Dell → Gmail)  
  - **QUIC Handshake packet (equivalent to TLS "Finished")**
  - **Contains ACK for previous handshake data**
  - **Padding length: 1040 bytes (used for security reasons)**  
  - 🔸 **Important Warning:**  
    - Wireshark **failed to decrypt QUIC traffic** because session secrets were not available.

- **Frame 51** (Gmail → Dell)  
  - **QUIC Short Header packet**
  - **This signals encryption is fully established**
  - **From this point onward, all Gmail communication is encrypted**

---

## **Step 3: Conclusion of QUIC Encryption Establishment**
After analyzing the QUIC handshake, we confirmed that:
✅ Gmail **does not use traditional TLS over TCP** for encryption in this session.  
✅ Instead, Gmail **uses QUIC over UDP (Port 443)**, which embeds encryption into its transport layer.  
✅ The **final step** in the handshake was **Frame 51**, where Gmail sent a **QUIC Short Header packet**, signaling that the session was fully encrypted.  
✅ At this point, all further email transmission data was **fully encrypted and unreadable** in Wireshark.

---

## **Observations & Key Takeaways**
### **1️⃣ Why Gmail Uses QUIC Instead of TLS Over TCP**
- **QUIC reduces latency** by eliminating the traditional multi-step handshake.
- **Combines transport and encryption setup** into a single flow.
- **Designed for fast and secure communication over unreliable networks** (improves Gmail’s performance).

### **2️⃣ Implications for Network Security Analysis**
- **Traditional TLS handshakes will not appear in QUIC traffic**—instead, encryption negotiation is embedded inside QUIC’s Crypto Frames.
- **Wireshark cannot decrypt QUIC traffic by default**, because QUIC encrypts more of the handshake than TLS 1.2/1.3.
- **For deeper QUIC analysis, session secrets must be extracted from the browser or system logs** (requires specialized tools).

---

## **Next Steps: Capturing Email Transmission from MacBook (Safari) to Dell (Chrome)**
Now that we have analyzed **encryption establishment for an email sent from Dell → Gmail**, we will perform a **similar capture for an email sent from the MacBook (Safari) to Gmail**, and compare:
1. **Does macOS Safari use QUIC, or does it fall back to TLS over TCP?**
2. **Are the Gmail servers and IP addresses different?**
3. **Does the QUIC handshake process differ between macOS and Windows?**

---


# **Nmap Scan Analysis: Gmail’s Mail Server (`smtp.gmail.com`)**
## **Objective**
To analyze Gmail’s email infrastructure by scanning its mail server (`smtp.gmail.com`) from the **Dell (Windows 11)** using Nmap.  
This scan provides insights into **open ports, encryption protocols, and security measures** in place.

---

## **1️⃣ Target Host Information**
- **Scanned Host:** `smtp.gmail.com`
- **IPv4 Address:** `74.125.137.108`
- **IPv6 Address (Not Scanned):** `2607:f8b0:4023:c03::6d`
- **Response Time:** `0.087s latency`
- **Reverse DNS:** `dy-in-f108.1e100.net`

---

## **2️⃣ Open Ports and Services**
| **Port** | **Protocol** | **Description** | **State** |
|----------|-------------|----------------|------------|
| **25** | **SMTP** | Standard SMTP (Mail Relay) | 🔴 **Filtered** |
| **465** | **SMTPS** | Secure SMTP over TLS | 🟢 **Open** |
| **587** | **Submission** | SMTP with StartTLS | 🟢 **Open** |
| **993** | **IMAPS** | Secure IMAP (Incoming Mail) | 🟢 **Open** |
| **995** | **POP3S** | Secure POP3 (Legacy Mail Retrieval) | 🟢 **Open** |

**Observations:**
- **Port 25 is blocked** for external use, preventing unauthorized email relay.
- **Secure email ports (SMTPS, IMAPS, POP3S) are open**, requiring authentication and encryption.
- **Gmail supports StartTLS on Port 587**, ensuring encryption can be added to plain-text connections.

---

## **3️⃣ TLS Encryption & Cipher Suite Analysis**
Gmail’s mail server **supports multiple TLS versions and ciphers**.

### **✅ Strong Security Measures**
- ✅ **TLS 1.3 is supported** (Most secure)
- ✅ **Elliptic Curve Ciphers (ECDHE) are used** for forward secrecy
- ✅ **Client-side cipher preference for TLS 1.2 & TLS 1.3** (security optimized)

### **⚠️ Security Weaknesses Detected**
- ❌ **TLS 1.0 and 1.1 are still enabled**, though deprecated.
- ❌ **3DES Cipher (`TLS_RSA_WITH_3DES_EDE_CBC_SHA`) is present**, which is vulnerable to the **SWEET32 attack**.
- ❌ **Server prefers weaker ciphers on TLS 1.0/1.1**, meaning legacy clients could be at risk.

---

## **4️⃣ Security & Compliance Observations**
- ✅ **Gmail enforces TLS for email transport**, securing data in transit.
- ❌ **Deprecated TLS 1.0/1.1 are still supported**, increasing legacy risk.
- ✅ **Gmail does not allow open SMTP relay on Port 25**, reducing spam risks.
- ✅ **Google prioritizes strong encryption for modern email clients**.

---

## **5️⃣ Next Steps: Run the Nmap Scan from the MacBook**
Now that we’ve analyzed Gmail’s mail server from the **Dell (Windows 11)**, we will repeat this scan **from the MacBook (Safari)** to:
- Compare **port availability** from a macOS environment.
- Check if Gmail presents **different encryption options based on the device**.
- Identify any **variation in TLS handshake behavior**.



# **Aligning Wireshark with Nmap: Ensuring We Scan the Right Protocol (IPv4 vs. IPv6)**
## **Objective**
To ensure that our **Nmap scanning aligns with real-world network traffic**, we need to **match the protocol (IPv4 or IPv6) used in Wireshark to our Nmap scan**. This ensures we **do not miss critical security insights**.

---

## **1️⃣ The Mistake We Made: Assuming IPv4 Without Checking Wireshark**
When we ran our **first Nmap scan**, we did not consider whether **Wireshark captured IPv4 or IPv6 traffic**.  

📌 **What We Did Wrong:**
- We **ran the default Nmap scan (`nmap smtp.gmail.com`)**, which scanned **only IPv4**.
- **But Wireshark had already shown that Gmail’s mail server was using IPv6 for our email traffic**.
- This meant that **our first scan did not match the real network behavior we observed**.

📌 **Lesson Learned:**
- **Always check Wireshark first to determine if the traffic is IPv4 or IPv6.**
- **Run an IPv6 Nmap scan (`nmap -6`) if Wireshark shows IPv6.**
- **Run an IPv4 Nmap scan if Wireshark shows IPv4.**

By **not checking Wireshark first**, we **almost missed** analyzing Gmail’s IPv6 security settings.

---

## **2️⃣ Why This Matters: The Risk of Only Scanning IPv4**
If we **only check IPv4 security**, we might **miss security weaknesses in IPv6 configurations**.

📌 **Common Organizational Mistake:**
- Many **organizations focus security efforts on IPv4 configurations**.
- **IPv6 is often less monitored**, leading to **accidental security gaps**.
- Example: An organization **disables weak ciphers on IPv4** but **forgets to disable them on IPv6**.

📌 **How Attackers Exploit This:**
- Some **attackers only scan IPv4** and assume the system is secure.
- A **more thorough attacker** scans **both IPv4 and IPv6** and **may find weaker security settings in IPv6**.

🚨 **Example Security Risk:**  
- An organization **blocks TLS 1.0 on IPv4** but **leaves it enabled on IPv6**.  
- Attackers **connect over IPv6** and **force outdated encryption**, bypassing security policies.

---

## **3️⃣ The Correct Workflow: Wireshark + Nmap Together**
To **ensure we always scan the correct protocol**, we should follow this workflow:

| **Step** | **Action** | **Tool Used** |
|----------|------------|--------------|
| **1️⃣ Capture network traffic** | Open Wireshark and analyze email transmission | **Wireshark** |
| **2️⃣ Identify IP protocol** | Check if traffic is **IPv4 or IPv6** | **Wireshark** |
| **3️⃣ Run Nmap scan matching Wireshark’s traffic** | If Wireshark shows IPv4 → Run `nmap smtp.gmail.com` <br> If Wireshark shows IPv6 → Run `nmap -6 smtp.gmail.com` | **Nmap** |
| **4️⃣ Compare results** | Check for encryption differences between IPv4 and IPv6 | **Nmap + Wireshark** |

📌 **Key Takeaway:**  
🚀 **Always let Wireshark guide your Nmap scan—never assume IPv4 unless traffic confirms it.**  
🚀 **By following this approach, we ensure that our security analysis is accurate and complete.**


# **Nmap IPv6 Scan Analysis: Gmail’s Mail Server (`smtp.gmail.com`)**
## **Objective**
To compare Gmail’s mail server security configurations between **IPv4 and IPv6** by scanning its **IPv6 address (`2607:f8b0:4023:c03::6d`)** using Nmap.  
This ensures Gmail applies **consistent security policies across both IP versions.**

---

## **1️⃣ IPv6 Scan Summary**
- **Scanned Host:** `smtp.gmail.com`
- **IPv6 Address:** `2607:f8b0:4023:c03::6d`
- **Response Time:** `0.041s` (Faster than IPv4: `0.087s`)
- **Reverse DNS:** `dy-in-f109.1e100.net`

---

## **2️⃣ Open Ports and Services (IPv6)**
| **Port** | **Protocol** | **Description** | **State** |
|----------|-------------|----------------|------------|
| **25** | **SMTP** | Standard SMTP (Mail Relay) | 🔴 **Filtered** |
| **465** | **SMTPS** | Secure SMTP over TLS | 🟢 **Open** |
| **587** | **Submission** | SMTP with StartTLS | 🟢 **Open** |
| **993** | **IMAPS** | Secure IMAP (Incoming Mail) | 🟢 **Open** |
| **995** | **POP3S** | Secure POP3 (Legacy Mail Retrieval) | 🟢 **Open** |

**Observations:**
- **Identical open ports to IPv4.**
- **Port 25 remains blocked for external users**, preventing spam and unauthorized relay.

---

## **3️⃣ TLS Encryption & Cipher Suite Analysis (IPv6)**
Gmail’s IPv6 mail server **supports the same encryption as IPv4**.

### **✅ Secure TLS Support**
- ✅ **TLS 1.3 supported** (Most secure, preferred for modern clients)
- ✅ **Elliptic Curve Ciphers (ECDHE) used** for forward secrecy
- ✅ **Client-side cipher preference for TLS 1.2 & TLS 1.3**

### **⚠️ Security Weaknesses Detected**
- ❌ **TLS 1.0 and 1.1 are still enabled**, though deprecated.
- ❌ **3DES Cipher (`TLS_RSA_WITH_3DES_EDE_CBC_SHA`) is present**, which is vulnerable to the **SWEET32 attack**.
- ❌ **Server prefers weaker ciphers on TLS 1.0/1.1**, meaning legacy clients could still connect with outdated security.

---

## **4️⃣ Key Takeaways: IPv4 vs. IPv6 Security Comparison**
| **Category** | **IPv4 (`74.125.137.108`)** | **IPv6 (`2607:f8b0:4023:c03::6d`)** | **Differences?** |
|-------------|------------------------|------------------------|-----------------|
| **Open Ports** | Same | Same | ❌ **No Difference** |
| **TLS Versions** | TLS 1.0–1.3 | TLS 1.0–1.3 | ❌ **No Difference** |
| **Weak Ciphers** | 3DES present | 3DES present | ❌ **No Difference** |
| **Response Time** | `0.087s` | `0.041s` | ✅ **IPv6 was faster** |

📌 **Conclusion:** Gmail enforces the **same security policies** across IPv4 and IPv6, with **no configuration gaps.**  
📌 **Next Step:** Capture **Wireshark traffic for MacBook → Dell email transmission**.


# **Final Summary: Nmap & Wireshark Email Security Analysis (Dell → Gmail)**

## **Objective**
This project analyzed **email transmission security using Nmap & Wireshark**, focusing on:
- **DNS resolution of Gmail’s mail servers**
- **TLS and QUIC encryption behavior**
- **Nmap scanning of Gmail’s SMTP/IMAP/POP3 servers**
- **IPv4 vs. IPv6 security settings**
- **Network security misconfigurations and potential attack vectors**

---

## **1️⃣ Key Findings**
### **🟢 DNS Resolution Analysis**
- Gmail’s mail server (`smtp.gmail.com`) resolved via DNS queries.
- The MacBook → Gmail resolution step was **not fully captured** due to network limitations.

### **🟢 Email Encryption: TLS vs. QUIC**
- Gmail’s web interface (browser-based) used **QUIC** encryption.
- Gmail’s mail servers (SMTP/IMAP) used **TLS 1.0, 1.1, 1.2, and 1.3**.
- **TLS 1.0 & 1.1 remain enabled, despite security deprecations**.

### **🟢 Nmap Scan of Gmail’s Mail Servers**
- **IPv4 and IPv6 scans revealed identical encryption settings.**
- **Legacy ciphers (3DES) were still present**, though unlikely used.
- **IPv6 was prioritized in our network, leading to an initial oversight in scanning.**

---

## **2️⃣ Lessons Learned**
✅ **Ensure Wireshark and Nmap scans are aligned to the correct IP version (IPv4 vs. IPv6).**  
✅ **QUIC vs. TLS differences impact visibility in packet captures.**  
✅ **Nmap confirms security policies that Wireshark cannot directly observe.**  
✅ **Capturing MacBook traffic from the Dell was not straightforward and would require a different setup.**  

---

## **3️⃣ Conclusion: Why We Stopped the MacBook Capture**
📌 **We determined that setting up MacBook traffic capture would require too much time compared to the value gained.**  
📌 **Instead of continuing with complex Wireshark configurations, we decided to conclude this project and start a new one with a fresh perspective.**  

🚀 **Next Steps: Start a new project with a different cybersecurity focus to explore another key area!**
